In [1]:
!pip install secretsharing
!pip install argon2_cffi
!pip install passlib
!pip install pycrypto

  Running setup.py bdist_wheel for secretsharing ... - done
  Stored in directory: /root/.cache/pip/wheels/09/c6/4a/c97772071806a92c6e60a6aa5811dfb25c909ca29f37b03127
Successfully built secretsharing
    100% |████████████████████████████████| 102kB 3.7MB/s 
    100% |████████████████████████████████| 409kB 20.7MB/s 
    100% |████████████████████████████████| 163kB 27.6MB/s 
  Running setup.py bdist_wheel for pycparser ... - \ done
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built pycparser
    100% |████████████████████████████████| 501kB 19.5MB/s 
    100% |████████████████████████████████| 450kB 13.5MB/s 
  Running setup.py bdist_wheel for pycrypto ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c

In [0]:
import os,sys,argparse,tempfile,shutil

# notes on secretsharing module below:
# - sudo -H pip install secret-sharing is needed first
# - secretsharing uses /dev/random by default, which is slow as it
#   gathers entropy from OS events - that's not only slow, but can
#   also frequently block, to get around this edit the source and
#   change it to use /dev/urandom which won't block
#   source to edit for me was:
#   /usr/local/lib/python2.7/dist-packages/secretsharing/entropy.py  
import secretsharing as sss

# for JSON output
#import jsonpickle # install via  "$ sudo pip install -U jsonpickle"

# for hashing passwords
from hashlib import sha256
import hashlib
# needed for these: sudo -H pip install passlib argon2_cffi
from passlib.hash import pbkdf2_sha256,argon2,sha512_crypt,sha1_crypt

# for non-security sensitive random numbers
from random import randrange

# for encrypting you need: sudo -H pip install pycrypto
import base64
from Crypto.Cipher import AES
from Crypto import Random
import json

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
#drive/My Drive/test/level1.json
#drive/My Drive/group_prac/code/00093_chapeaua.json
with open('drive/My Drive/group_prac/Layer_9/JSON/level9.json') as f:
    data = json.load(f)

raw_cypher=data["ciphertext"]
hashes=data["hashes"]
shares=data["shares"]
easteregg=data["easteregg"]
print(hashes)
print(len(hashes))

print(data)
print(easteregg)

[u'$6$rounds=656000$HbeFMkrWw3d9tIKC$0.7kw/SQNsuMXf2n3tB7tIxr26KnXx7oSF0HUZ0IS8mEqZGv9mR6tPNFWFRDgyWmdNDLcMU/sV275Mq/fuQXZ/', u'$6$rounds=656000$mfhynIqNgdwh6g/k$hh83VJRjgKc94DRlVUwNYESmMreX5vwGGfg6KO.As/DU9NSp7X1FpgywU51.hj6pnDqwluk7XDXjhdI/0qA3F1', u'$argon2i$v=19$m=102400,t=2,p=8$Wct5z5kTYuwdw5hzTgnBGA$wT8elg8XqZiDtp2YBFUq7w', u'$argon2i$v=19$m=102400,t=2,p=8$T6n1XgthbG1tjfF+D6E0hg$HrKPnF4VzPSutTsHIgvogg', u'$argon2i$v=19$m=102400,t=2,p=8$YIwxxhgDYKy1FqL0fm+NMQ$KRsvXsfepLJ3l4YMsZR5Tg', u'$sha1$480000$ChV0HEXF$xwe1pGlusxogHjYSDpGEmrIJCIFy', u'$argon2i$v=19$m=102400,t=2,p=8$+F9rbc2ZM4Yw5ry3FsLYew$BIWiQkOMhH9VyN/XBzmPUw', u'$argon2i$v=19$m=102400,t=2,p=8$YMw5p3ROKaWU8r6X8n5vjQ$yosVUdk+qUp/zK8Jx28pXA', u'$pbkdf2-sha256$29000$z3mvFaJU6t0b4xxjbI2RUg$8pEwBdXP.FeA1qGMiLoD8LmRPoXPVXfKiwan8ICHbec', u'$pbkdf2-sha256$29000$0poTIsT43zuntDZGqFWKEQ$71.fdI1v9dZHJdeZdhpqy7m8jDcq9VEZ/QRp3wH10As', u'$argon2i$v=19$m=102400,t=2,p=8$2fuf834P4TznPAcAwDgHQA$LR8lqQnUxOlX9vJE7DQ0Rg', u'$sha1$480000$Lxtt3oab$

In [5]:
#drive/My Drive/test/pass.potfile
with open("drive/My Drive/group_prac/Layer_9/potfile/potfile_pbkdf2_L9.txt") as f:
    passwords = f.readlines()
passwords = [x.strip() for x in passwords]
print passwords

['$pbkdf2-sha256$29000$TImR8t47R8iZU4rR.t/7Pw$VLehbYo4pdPo6jCNVUsuOiZXHu3CT/kbNMaQCvzAFS0:ADDELT', '$pbkdf2-sha256$29000$JSQkhBCCkLL2HoMQopRS6g$kD2KJ7zAyUHUJbr1iEBIrdNl8QAtKTJOqq07gaOhkIE:SOLOKYRA', '$pbkdf2-sha256$29000$pVQKoZRyjjHGOEfofU9p7Q$Dmst77Y5e5gquA9TXd/Ab4pqXu4LGrdqMsKqXWooesc:TERZAH', '$pbkdf2-sha256$29000$7v3/H.O8d07pnXNuzdlb6w$EIDM.lYFEJ3KAGOMbN8IvaXLv0ENH3A6PhwrCNSqE7c:tali25o', '$pbkdf2-sha256$29000$glDq3ZvzPqfUOiek1JrTOg$ATSxg1u4BD.SXXKe0vfWVWiq.UO7VETziAZ6otNBPBo:V00LFICJ', '$pbkdf2-sha256$29000$SYkR4lwr5TyH0FrLOQdAaA$NP38Bf.SQ8RaS5WB/OT5gZI8WAf4.qxb1gYiEaT1AD0:0052658', '$pbkdf2-sha256$29000$/t.7txbC.H/vfQ.h1Nr7nw$Avop2/2w027MmhqlpmVOzlJNLIeTMlWMVB.z/6xcd8A:05884292', '$pbkdf2-sha256$29000$SMk5B6BUCuE8R0gpxRjDeA$PywXY2hHdE6sNb8P8UHdU3FviY.NKP4ecuzOTeBfDS4:15717869', '$pbkdf2-sha256$29000$TUmJ0RojxJgTQoiR8t4bIw$.RDzVo8BNfv2NA.zJUnwP5bIheaRYQeWHLmD8LgeNsM:1thujina', '$pbkdf2-sha256$29000$GuN8D0HoPSdEyNkboxSCsA$QfkT6EDszxt/7LCMzV7itoIbYYthwxXErloQXffYTTE:21923700', '$pbk

In [0]:
inds=[]
passwords_list=[]
#GPV: changing the code to account for plaintexts that include ':'
for ps in range(len(passwords)):
  #print(passwords[ps].split(":")[0])
  pos = passwords[ps].find(":")
  print(passwords[ps][0 : pos])
  #inds.append(hashes.index(passwords[ps].split(":")[0]))
  inds.append(hashes.index(passwords[ps][0 : pos]))
  #passwords_list.append(passwords[ps].split(":")[1])
  passwords_list.append(passwords[ps][pos+1:])

print(len(inds))
print(len(passwords_list))

$pbkdf2-sha256$29000$TImR8t47R8iZU4rR.t/7Pw$VLehbYo4pdPo6jCNVUsuOiZXHu3CT/kbNMaQCvzAFS0
$pbkdf2-sha256$29000$JSQkhBCCkLL2HoMQopRS6g$kD2KJ7zAyUHUJbr1iEBIrdNl8QAtKTJOqq07gaOhkIE
$pbkdf2-sha256$29000$pVQKoZRyjjHGOEfofU9p7Q$Dmst77Y5e5gquA9TXd/Ab4pqXu4LGrdqMsKqXWooesc
$pbkdf2-sha256$29000$7v3/H.O8d07pnXNuzdlb6w$EIDM.lYFEJ3KAGOMbN8IvaXLv0ENH3A6PhwrCNSqE7c
$pbkdf2-sha256$29000$glDq3ZvzPqfUOiek1JrTOg$ATSxg1u4BD.SXXKe0vfWVWiq.UO7VETziAZ6otNBPBo
$pbkdf2-sha256$29000$SYkR4lwr5TyH0FrLOQdAaA$NP38Bf.SQ8RaS5WB/OT5gZI8WAf4.qxb1gYiEaT1AD0
$pbkdf2-sha256$29000$/t.7txbC.H/vfQ.h1Nr7nw$Avop2/2w027MmhqlpmVOzlJNLIeTMlWMVB.z/6xcd8A
$pbkdf2-sha256$29000$SMk5B6BUCuE8R0gpxRjDeA$PywXY2hHdE6sNb8P8UHdU3FviY.NKP4ecuzOTeBfDS4
$pbkdf2-sha256$29000$TUmJ0RojxJgTQoiR8t4bIw$.RDzVo8BNfv2NA.zJUnwP5bIheaRYQeWHLmD8LgeNsM
$pbkdf2-sha256$29000$GuN8D0HoPSdEyNkboxSCsA$QfkT6EDszxt/7LCMzV7itoIbYYthwxXErloQXffYTTE
$pbkdf2-sha256$29000$g1AqBaDUmhMiRChFyPkfww$mCJnsVIlIf4HGN862Tk8/QW6iuneZK4vP.TVGh9/2Lk
$pbkdf2-sha256$29000$l1IKwbgXYmw

In [0]:

shares_list=[]

for x in shares:
  shares_list.append(str(x))

print((passwords_list))
print((inds))
print(len(passwords_list))


['ADDELT', 'SOLOKYRA', 'TERZAH', 'tali25o', 'V00LFICJ', '0052658', '05884292', '15717869', '1thujina', '21923700', '26604332', '304953', '52768637', '79892125', '86287210', '87117078', '8933288', '91514743', 'Ablatuur', 'cakesux7', 'ertsryk8', 'GNIbaa', 'halasaf', 'itai195', 'Jakemin', 'Kumaaro6', 'pear912', 'raryduct', 'Ruessits', 'semstad0', 'spessare', 'tovamman', 'Turdored', 'Zhashui']
[49, 18, 43, 140, 67, 58, 155, 161, 88, 93, 109, 141, 76, 92, 39, 29, 142, 41, 110, 37, 34, 108, 152, 169, 127, 16, 148, 153, 23, 104, 114, 9, 137, 144]
34


In [0]:
import os,sys,argparse,tempfile,shutil 

# notes on secretsharing module below:
# - sudo -H pip install secret-sharing is needed first
# - secretsharing uses /dev/random by default, which is slow as it
#   gathers entropy from OS events - that's not only slow, but can
#   also frequently block, to get around this edit the source and
#   change it to use /dev/urandom which won't block
#   source to edit for me was:
#   /usr/local/lib/python2.7/dist-packages/secretsharing/entropy.py  
import secretsharing as sss

# for hashing passwords
from hashlib import sha256

# for encrypting you need: sudo -H pip install pycrypto
import base64
from Crypto.Cipher import AES
from Crypto import Random
BLOCK_SIZE = 16
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)

# our cs7ns1-specific functions for shamir-like sharing
def pxor(pwd,share):
  '''
    XOR a hashed password into a Shamir-share
    1st few chars of share are index, then "-" then hexdigits
    we'll return the same index, then "-" then xor(hexdigits,sha256(pwd))
    we truncate the sha256(pwd) to if the hexdigits are shorter
    we left pad the sha256(pwd) with zeros if the hexdigits are longer
    we left pad the output with zeros to the full length we xor'd
  '''
  words=share.split("-")
  hexshare=words[1]
  slen=len(hexshare)
  hashpwd=sha256(pwd).hexdigest()
  hlen=len(hashpwd)
  outlen=0
  if slen<hlen:
      outlen=slen
      hashpwd=hashpwd[0:outlen]
  elif slen>hlen:
      outlen=slen
      hashpwd=hashpwd.zfill(outlen)
  else:
      outlen=hlen
  xorvalue=int(hexshare, 16) ^ int(hashpwd, 16) # convert to integers and xor 
  paddedresult='{:x}'.format(xorvalue)          # convert back to hex
  paddedresult=paddedresult.zfill(outlen)       # pad left
  result=words[0]+"-"+paddedresult              # put index back
  return result

def pwds_shares_to_secret(kpwds,kinds,diffs):
  '''
      take k passwords, indices of those, and the "public" shares and 
      recover shamir secret
  '''
  shares=[]
  for i in range(0,len(kpwds)):
    shares.append(pxor(kpwds[i],diffs[kinds[i]]))
  secret=sss.SecretSharer.recover_secret(shares)
  return secret

#pwds = ['hello', 'bye', 'hi there']
#diffs = ['1-af52d4a9c32', '2-af52d4a9c32','3-af52d4a9c32']
#inds = [0, 1, 2]
temp=[]
#GPV: changing the loop to include the last plaintext when generating the key
for i in range(1,len(passwords_list) + 1):
#for i in range(1,len(passwords_list)):
  key=pwds_shares_to_secret(passwords_list[0:i], inds[0:i], shares_list)
  print(i,"----",key)
  temp.append(key)
print((key))

(1, '----', 'be434bee3d5b9818518884e76387b9a')
(2, '----', '2dec9fc15b42105ff8fc4e6e265e9054')
(3, '----', '6f281d840b9974044433a0933d93803b')
(4, '----', '45d4b0af2bf9377b4b82fc2cf5deea26')
(5, '----', '26e27fb8f645a5354178f54084658c30')
(6, '----', '257efc75357a53818643711614960203')
(7, '----', '106891304a2b856ee0b57bfab84f015c')
(8, '----', '3d509e437907a79736c76ab677facde4')
(9, '----', '344e24bb3aab8be3de24499e48838fa3')
(10, '----', '2f46e01a335b85339dbf61803d0da61b')
(11, '----', '7e66df7af2c5341ec4f585593add93a2')
(12, '----', '190df2cd0e9d207c3388838f28f60c66')
(13, '----', '6a12a68aa710afa83c310d173e55ef59')
(14, '----', '599156d61125a0f6fa945245266cc51d')
(15, '----', '48cc604b9beeb5978d43bf7dcf932ce7')
(16, '----', '27a090c82cf82bd44e2aaabdf242de62')
(17, '----', '4668568260135b6c5c6f61ce4fb45093')
(18, '----', '69eddf4d79f2cabe0b1c5596b958b296')
(19, '----', '2fe62f3cee5804d21b2b80a47a0577ab')
(20, '----', '242027710c53da99d94bab557706ed9e')
(21, '----', '35031629af5d739a

In [0]:
unpad = lambda s: s[:-ord(s[len(s) - 1:])]
def decrypt1( key, enc ):
    enc = base64.b64decode(enc)
    iv = enc[:16]
    cipher = AES.new(key, AES.MODE_CBC, iv )
    return unpad(cipher.decrypt( enc[16:] ))

In [0]:
x=decrypt1(key.zfill(32).decode('hex'),raw_cypher)
#encode("utf-8")
print type(x)
print(x)
#print x.zfill(32).decode('hex')

#decrypt(raw_cypher,key.zfill(32).decode('hex'))

<type 'str'>
j��kAO�+��W���k�Iꗵ��+lF��K���ꥲ�B�D��\�ch&�s�����aaw��B~��4�����'Q���UMO�B��A��ƫ
Wl5\��iop���7�T�k	���x��շI�R���ѐ_4pBpb;�=��`ق�CS*�W�̓�
W�&쯨�t�G���;t�r%�B[kE�����g��vqa������d�H5�]�ZWAW��J�QC��	����O���Xκ�޷W)E�a���?"�
���Wo�{�⧀��[�*rF&��6�K��-�<ֆ���':��pJ�������ظ�W"�'���n��O�k�x٤A
[���a�1�wa���4�l/Dh�� 6��c�*��\�rŁ2`�a�7���,����M����1������Wسk�_�tZ�����{虩�)�V5�7}��l�lxr��)n�j��pod8�ŷ+�B%����;��L���[ �#�8r��i���GH����u�!7.��;:dܘY�|�uk{�.��K/ g:�v
F��N��¨:<.����R>��z�/����K�dm攱�"�۫cp�'���.)������нL
�O�k���^��8��"*��"��ʻ�3�u�BA��F9�u ��`>{9����|[C��+�!�ͻ.�i�؎�]���عAҽF��S��lz^ܻwG�XM
l�ȃ=��w���=b��U�D\��( �+Ѓ�4H
�'F�~�����od�_�`��\0�:��7�Y��o�9=�b�6V���8M� �`E�H�'g61��_�3+��G��JT��&�Z�muQU� j�AܴR4<�䜐u�t�"�wR2w
��՛|}�פȲў'����1^�p��j!Lķ]��D���4*� �:t�H�g���_��y��ƪ>�6g��$$
5�������{a1�7F�/�����:\�*9J�1��m6Q�r��0�-�}�bt�����ٙ���Ֆ���K�-)7���8F��~���c���l��kA��t�W�]��í�@������Q�U����.J�

In [0]:
with open('drive/My Drive/group_prac/Layer_7/JSON/level7.json', 'w') as outfile:
    for n in x:
        outfile.writelines(n)

In [0]:
from google.colab import files
with open('drive/My Drive/group_prac/Layer_7/JSON/level7.json') as f2:
    data2 = json.load(f2)

raw_cypher=data2["ciphertext"]
new_hashes=data2["hashes"]
new_shares=data2["shares"]
print(new_hashes)

with open('drive/My Drive/group_prac/Layer_7/hashes/new_hashes.txt', 'w') as f:
  for x in new_hashes:
    f.writelines("%s\n" % str(x))
print len(new_hashes)


[u'$pbkdf2-sha256$29000$vvdey/lf6937/9/7v5dSKg$LYxP/Bx1mXuo8CapFp0ayKdZKBYsuthxZX1VaTqvCzU', u'$sha1$480000$clOYrHve$rxzu8DVgtEn2HC6CgcNgmT6HfjLc', u'$sha1$480000$..vzve1b$TUp9tuoOIK2sESr9ItgkcxQWjgiG', u'$argon2i$v=19$m=102400,t=2,p=8$/j8HYExp7X1vDaGU8h7DuA$GyMFqo8x4ASRTeDw7iSvHg', u'$sha1$480000$AQf.m9BG$OwbzDD9.QgAqSvU1NYCPstSXy1CT', u'$pbkdf2-sha256$29000$bc05J.Sc834PIQTAGCNEyA$H5OETEzDIOJ.UfKUP1Y01DbTlx3DJ3GPUdL//6tutWI', u'$6$rounds=656000$LOt7P.hpz.vY4V9d$pAO0XoL37zApi2kMIT9v8JjUXazPIGmFVtHZMhfzNyA5h2SuD544q4bi7MvH/1bI0go7Y.YHTAitaQsVhrPr./', u'$sha1$480000$eecLApy7$kfDYaKMqsph1U4LoTYMs2xf3EKTT', u'$argon2i$v=19$m=102400,t=2,p=8$5rzX2rtXqnVOqdWakxKi1A$HBTVOAl7T49tDcjtaFr1OQ', u'$argon2i$v=19$m=102400,t=2,p=8$oTSGkJJSqjWmFAJAyDmH8A$TXlotQbXSEzMFfSchzXDNw', u'$pbkdf2-sha256$29000$ei9lLKVUam2tNab0fm9tzQ$fKuXeuqSYXnTabFkQDvmAzx4WYa3PjWDoc9dPaoVccg', u'$argon2i$v=19$m=102400,t=2,p=8$1vofg/D+X4uxNqbUGuP8Xw$EMNKzfcS4eiEtLhAf2/axg', u'$sha1$480000$mCWOVFo0$J.RkEygZxfs2GGE.KEPL8Me6/zC/',

In [0]:
from google.colab import files

with open('hashes.txt', 'w') as f:
  for x in hashes:
    f.writelines("%s\n" % str(x))

files.download('hashes.txt')